In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [4]:
## self
import config
from preprocess import *
from model import *
from utils import *
from train import *

from transformers import BertModel
import torch.nn as nn
import math # for sqrt

In [6]:
dev_path = config.DEV_MA_FILE
print(dev_path)

/work/2020-IIS-NLU-internship/MNLI/data/multinli_1.0/multinli_1.0_dev_matched.jsonl


In [6]:
tokenizer = BertTokenizer.from_pretrained(config.BERT_EMBEDDING) # for tensor to sent usage

In [8]:
dev_set = MNLI_Raw_Dataset(dev_path, mode="develop")
print(len(dev_set))

9815


In [9]:
# quick test for trainer
trainer_test = CrossBERT_Trainer()
model, stats = trainer_test.train(train_set=dev_set[:5], dev_set=dev_set[5:10])

trainer device is cuda
no log_stream specified, logger printing to default path/work/2020-IIS-NLU-internship/MNLI/param/cross-bert-comp-maxpool/train_log.txt
no stat_stream specified, pure statistics printing to default path/work/2020-IIS-NLU-internship/MNLI/param/cross-bert-comp-maxpool/stat.jsonl


100%|██████████| 1/1 [00:00<00:00, 32.63it/s]
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
100%|██████████| 1/1 [00:00<00:00, 31.77it/s]


In [9]:
dev_loader = DataLoader(
    dataset=dev_set, 
    batch_size=config.BATCH_SIZE,
    shuffle=True,  
    collate_fn=create_mini_batch
)
if config.DEBUG:
    #print(next(iter(dev_loader)))
    pass

In [10]:
if config.DEBUG:
    print(tensor_to_sent(next(iter(dev_loader))[config.h_field]['input_ids'][0], tokenizer))
    print(type(next(iter(dev_loader))[config.h_field]['input_ids'][0]))

[CLS] so that ' s what you could ' ve done if things had been different ? [SEP] [PAD] [PAD] [PAD]
<class 'torch.Tensor'>


In [10]:
model = CrossBERTModel()

unkown bert model choice, init with config.BERT_EMBEDDING


In [11]:
test_batch = next(iter(dev_loader))

In [12]:
print(len(test_batch[config.h_field]["input_ids"]))
print(len(test_batch[config.h_field]["input_ids"][5]))
print(len(test_batch[config.p_field]["input_ids"]))
print(len(test_batch[config.p_field]["input_ids"][5]))

8
22
8
55


In [13]:
model._predict_score(test_batch)
model(test_batch)

(tensor(0.6720, grad_fn=<BinaryCrossEntropyWithLogitsBackward>),
 tensor([[-0.4289, -2.0630, -0.7061],
         [ 0.3484, -2.0866, -0.6215],
         [-0.3850, -1.5685, -1.5595],
         [-0.0731, -1.2159, -2.2720],
         [ 0.2064, -1.2605, -1.0619],
         [-0.1231, -0.0031, -1.9876],
         [-0.7452, -1.1626, -1.0438],
         [-0.0334, -1.7249, -1.8074]], grad_fn=<SqueezeBackward1>))

In [16]:
trainer = CrossBERT_Trainer(model=model)

In [19]:
model

CrossBERTModel(
  (bert_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

In [21]:
#aa = trainer.eval_model_when_training(model.to('cuda'), dev_set)
print(aa['confusion_matrix'])

NameError: name 'aa' is not defined

In [22]:
if False:
    model_to_save = model
    import os
    save_model_path = config.PARAM_PATH / "test"
    if not os.path.isdir(save_model_path):
        os.mkdir(save_model_path)
    epoch_stat = aa
    torch.save(model_to_save.state_dict(),
                str(save_model_path / "model_epoch{0}_precision:{1:.3f}_recall:{2:.3f}_f1:{3:.3f}_acc:{4:.3f}.m".
                format(0, epoch_stat['report']['macro avg']['precision'], epoch_stat['report']['macro avg']['recall'], epoch_stat['report']['macro avg']['f1-score'], epoch_stat['report']['accuracy'])))

In [23]:
if False:
    model_load = CrossBERTModel()
    checkpoint = torch.load(save_model_path / "model_epoch0_precision:0.333_recall:0.109_f1:0.164_acc:0.327.m")
    model_load.load_state_dict(checkpoint)

In [25]:
trainer = CrossBERT_Trainer()
model, stats = trainer.train(train_set=dev_set[:5], dev_set=dev_set[5:10])

  0%|          | 0/1 [00:00<?, ?it/s]

initializing new model
model in cuda? True
trained_model will be in /work/2020-IIS-NLU-internship/MNLI/param/CrossBERT_baseline
<generator object Module.named_parameters at 0x7fd6fbba6350>
start training ... 


100%|██████████| 1/1 [00:00<00:00, 28.96it/s]
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


lr = 0.000000
epoch 0 train_loss: 0.977
epoch 0 eval_precision: 0.167 eval_recall: 0.333 eval_f1: 0.222


100%|██████████| 1/1 [00:00<00:00, 25.42it/s]


...

lr = 0.000001
epoch 3 train_loss: 0.961
epoch 3 eval_precision: 0.278 eval_recall: 0.667 eval_f1: 0.389


100%|██████████| 1/1 [00:00<00:00, 25.31it/s]


lr = 0.000002
epoch 4 train_loss: 0.868
epoch 4 eval_precision: 0.111 eval_recall: 0.167 eval_f1: 0.133


100%|██████████| 1/1 [00:00<00:00, 25.54it/s]


lr = 0.000002
epoch 5 train_loss: 0.852
epoch 5 eval_precision: 0.111 eval_recall: 0.167 eval_f1: 0.133


In [ ]:
trainer = CrossBERT_Trainer()
model, stats = trainer.train() # use config sets

loading trainset from /work/2020-IIS-NLU-internship/MNLI/data/multinli_1.0/multinli_1.0_train.jsonl
loading devset from /work/2020-IIS-NLU-internship/MNLI/data/multinli_1.0/multinli_1.0_dev_matched.jsonl


  0%|          | 0/49088 [00:00<?, ?it/s]

initializing new model
model in cuda? True
trained_model will be in /work/2020-IIS-NLU-internship/MNLI/param/CrossBERT_baseline
<generator object Module.named_parameters at 0x7fd498c3b450>
start training ... 


  0%|          | 3/1227 [00:00<00:46, 26.24it/s]

lr = 0.000025
epoch 0 train_loss: 0.458


100%|██████████| 1227/1227 [00:48<00:00, 25.35it/s]


epoch 0 eval_precision: 0.708 eval_recall: 0.718 eval_f1: 0.709


 75%|███████▌  | 37013/49088 [1:16:06<25:22,  7.93it/s]  